# Importando dados



In [722]:
import pandas as pd
import numpy as np
from pathlib import Path

> Os dados são importados utilizando a `pathlib` a fim de evitar problemas de caminho de UNIX x Windows

In [723]:
dataPath = Path('Leukemya_data.csv').relative_to('.')
targetPath = Path('labels.csv').relative_to('.')
dados = pd.read_csv(dataPath, header = None)
resultados = pd.read_csv(targetPath, header = None)

# Dividir dados para treinamento

In [724]:
dados_conhecidos = dados.head(128)
dados_teste = dados.tail(50)

# Definição do Modelo

> Os treinos serão usados para treinar o `DecisionTreeClassifier` enquanto os testes serão utilizados para verifcar as estatísticas do modelo treinado (precisão, acurácia, etc)

In [725]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

x_treino, x_teste, y_treino, y_teste = train_test_split(dados_conhecidos, resultados, test_size=0.3, shuffle=False, stratify=None)
DT = DecisionTreeClassifier(random_state=128).fit(x_treino, y_treino)

# Análise do Modelo

### Importando

In [726]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve # Desnecesário
from sklearn.metrics import auc # Desnecesário

### Estatísticas do Modelo 

In [727]:
previsao = DT.predict(x_teste)
acuracia = accuracy_score(y_teste, previsao)
precisao = precision_score(y_teste, previsao)
mcc = matthews_corrcoef(y_teste, previsao)
tn, fp, fn, tp = confusion_matrix(y_teste, previsao).ravel()

print(f'Acurácia: {acuracia:0.3f}')
print(f'Precisão: {precisao:0.3f}')
print(f'Coeficiente de Matthes: {mcc:0.3f}')
print(f'Falsos Positivos: {fp}')
print(f'Falsos Negativos: {fn}')


Acurácia: 0.949
Precisão: 0.941
Coeficiente de Matthes: 0.820
Falsos Positivos: 0
Falsos Negativos: 2


# Resultado

> O Resultado é salvo no arquivo `Diagnostico.csv` apenas com os pacientes faltantes (50 últimos)

In [728]:
numero_diagnostico = pd.DataFrame({ "Diagnóstico": DT.predict(dados_teste)}, index=dados_teste.index, )
diagnostico = numero_diagnostico.applymap(lambda n: "Leucemia" if n == 2 else "Saudável")

pd.concat([dados_teste, diagnostico], axis=1).to_csv("Diagnostico.csv")